In [1]:
import pandas as pd
from openai import OpenAI
import openai
import tqdm
import datetime

In [2]:
def create_messages(text, emotion):
    return [
        {
            "role": "system",
            "content": "You are an analytical tool designed to identify rhetorical pathos in text. Argumentate which part of the sentence can be used to identify rhetorical pathos in text. Emotion is already provided. If you don't agree with the emotion found, return 'unclear'."
        },
        {
            "role": "user",
            "content": f"Analyze the following text: {text}. Emotion found in text: {emotion}"
        },
        {
            "role": "assistant",
            "content": f"Argumentate which part of the text indicates emotion found."
        }
    ]

In [ ]:
client = openai.OpenAI(base_url="http://localhost:1234/v1",  api_key="not-needed")
model_name = 'mixtral8x7bv01q6k'

path_to_csv = 'output/models/distilbert-base-uncased_2024-03-26_10-03/validated.csv'
path_to_model_dir = path_to_csv.rsplit('/', 1)[0]

df = pd.read_csv(path_to_csv, sep=',', header=0)

responses = []
for i, (txt, label, predicted) in tqdm.tqdm(df.iterrows()): # iterate through each text value of first df column
    completion = client.chat.completions.create(
      model="local-model", # this field is currently unused
      messages=create_messages(txt, predicted),
      temperature=0.7,
      max_tokens=100
    )
    responses.append(completion.choices[0].message.content) # append response to list
    

0it [00:00, ?it/s]

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

with open(f'{path_to_model_dir}/{model_name}_{timestamp}_message.txt', 'w') as f:
    f.write(str(create_messages('_','_')))

df.to_csv(f'{path_to_model_dir}/{model_name}_{timestamp}_validated_with_argumentation.csv', sep=',', index=False)